This is a fixed version of [@felipebihaiek](https://www.kaggle.com/felipebihaiek) [episode scraper](https://www.kaggle.com/felipebihaiek/google-football-episode-scraper-quick-fix), which returned a TypeError for None values.
The downloaded files are zipped at the end of the notebook. Make sure that you do not excess the memory limit by downloading too much files. You can change the first selected team by modifying the top_teams[i] i indice at the top of the second to last cell

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time

In [ ]:
MIN_FINAL_RATING = 1560 # top submission in a match must have reached this score
num_api_calls_today = 0
NUM_TEAMS = 1
EPISODES = 400

In [ ]:
BUFFER = 1

base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"

In [ ]:
# inital team list

r = requests.post(list_url, json = {"teamId":5673355}) # arbitrary ID, change to leading ID during challenge
rj = r.json()
teams_df = pd.DataFrame(rj['result']['teams'])

In [ ]:
teams_df.sort_values('publicLeaderboardRank', inplace = True)
# Selecting the top 10
teams_df = teams_df.iloc[:10,:]
teams_df

In [ ]:
def getTeamEpisodes(team_id,teams_df):

    r = requests.post(list_url, json = {"teamId":  int(team_id)})
    rj = r.json()

    # update teams list
    # global teams_df
    teams_df_new = pd.DataFrame(rj['result']['teams'])

    if len(teams_df.columns) == len(teams_df_new.columns) and (teams_df.columns == teams_df_new.columns).all():
        teams_df = pd.concat( (teams_df, teams_df_new.loc[[c for c in teams_df_new.index if c not in teams_df.index]] ) )
        teams_df.sort_values('publicLeaderboardRank', inplace = True)
    else:
        print('teams dataframe did not match')

    # make df
    team_episodes = pd.DataFrame(rj['result']['episodes'])
    team_episodes['avg_score'] = -1;

    agents = []
    for i in range(len(team_episodes)):
        agents.append(team_episodes['agents'].loc[i])
    not_none_agents = []
    for i in range(len(agents)):
        l = [a['updatedScore'] for a in agents[i] if a['updatedScore'] is not None]
        if l:
            not_none_agents.append(agents[i])
            
    for i in range(len(not_none_agents)):
        agents = not_none_agents[i]
        agent_scores = [a['updatedScore'] for a in agents if a['updatedScore'] is not None]
        team_episodes.loc[i, 'submissionId'] = max([a['submissionId'] for a in agents])
        team_episodes.loc[i, 'updatedScore'] = max([a['updatedScore'] for a in agents]) 
    
    team_episodes['final_score'] = team_episodes['updatedScore']
    team_episodes.sort_values('final_score', ascending = False, inplace=True)
    
    return rj, team_episodes

In [ ]:
def saveEpisode(epid, rj):
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})
        
    # save replay
    with open('{}.json'.format(epid), 'w') as f:
        f.write(re.json()['result']['replay'])

In [ ]:
global num_api_calls_today

pulled_teams = {}
pulled_episodes = []
start_time = datetime.datetime.now()
r = BUFFER;

In [ ]:
while num_api_calls_today < EPISODES:
    # pull team
    top_teams = [i for i in teams_df.id if i not in pulled_teams]
    if len(top_teams) > 0:
        team_id = top_teams[0]
    else:
        break;
        
    # get team data
    team_json, team_df = getTeamEpisodes(team_id,teams_df); r+=1;
    
    num_api_calls_today+=1
    
    print('{} games for {}'.format(len(team_df), teams_df.loc[teams_df.id == team_id].iloc[0].teamName))

    
    team_df = team_df[  (MIN_FINAL_RATING is None or (team_df.final_score > MIN_FINAL_RATING))]
    
    print('   {} in score range from {} submissions'.format(len(team_df), len(team_df.submissionId.unique() ) ) )
    
    team_df = team_df[~team_df.id.isin(pulled_episodes)]        
    print('      {} remain to be downloaded\n'.format(len(team_df)))
        
    # pull games
    target_team_games = int(np.ceil(EPISODES / NUM_TEAMS))
    if target_team_games + len(pulled_episodes) > EPISODES:
        target_team_games = EPISODES - len(pulled_episodes)
     
    pulled_teams[team_id] = 0
    
    i = 0
    while i < len(team_df) and pulled_teams[team_id] < target_team_games:
        epid = team_df.id.iloc[i]
        if not (epid in pulled_episodes):
            try:
                saveEpisode(epid, team_json); r+=1;
                num_api_calls_today+=1
            except:
                time.sleep(20)
                i+=1;
                continue;
                
            pulled_episodes.append(epid)
            pulled_teams[team_id] += 1
            try:
                size = os.path.getsize('{}.json'.format(epid)) / 1e6
                print(str(num_api_calls_today) + ': Saved Episode #{} @ {:.1f}MB'.format(epid, size))
            except:
                print('  file {}.json did not seem to save'.format(epid))    
            if r > (datetime.datetime.now() - start_time).seconds:
                time.sleep( r - (datetime.datetime.now() - start_time).seconds)
                

        i+=1;
    print(); print()

In [ ]:
# importing required modules 
from zipfile import ZipFile 
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def zip(): 
    # path to folder which needs to be zipped 
    directory = './'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths: 
        print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('my_python_files.zip','w') as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 
  
    print('All files zipped successfully!')         
  
zip()